In [28]:
import os
import json
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

def process_jsonl(file_path):
    completions = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                try:
                    data = json.loads(line.strip())
                    messages = data['body']['messages']

                    completion = client.chat.completions.create(
#                        model=data['body']['model'],
                        model='gpt-3.5-turbo',
                        response_format={"type": "json_object"},
                        messages=messages,
                        max_tokens=data['body']['max_tokens']
                    )

                    # Parse the JSON string within the completion response
                    json_response = json.loads(completion.choices[0].message.content)
                    completions.append(json_response)
                except json.JSONDecodeError:
                    print("Error decoding JSON")
                except Exception as e:
                    print(f"Error during API call: {e}")
    except IOError:
        print(f"Error opening or reading the file: {file_path}")

    return completions

def jsonl_concatenate(directory_path, output_file_path):
    """
    Concatenates all JSONL files in the specified directory into a single JSONL file.
    
    Args:
        directory_path (str): The path to the directory containing JSONL files.
        output_file_path (str): The path where the concatenated JSONL file will be saved.
    """
    jsonl_files = [f for f in os.listdir(directory_path) if f.endswith('.jsonl')]
    with open(output_file_path, 'w') as output_file:
        for file_name in jsonl_files:
            file_path = os.path.join(directory_path, file_name)
            with open(file_path, 'r') as file:
                for line in file:
                    output_file.write(line)

# Define the directory containing JSONL files and the path for the output file
directory_path = 'Datasets/oldtimey touringbot version 2/Svenska Dagbladet 1908-01-01-1908-12-31/KB Extracted Data'
output_file_path = 'Datasets/oldtimey touringbot version 2/Svenska Dagbladet 1908-01-01-1908-12-31/OpenAI Results/1908_SD.json'

# Call the function to concatenate JSONL files
jsonl_concatenate(directory_path, output_file_path)

# Process the concatenated JSONL file
results = process_jsonl(output_file_path)

# Open the output file in write mode to store the JSON array
with open(output_file_path, 'w') as f:
    f.write('[')  # Start the JSON array
    for index, result in enumerate(results):
        json.dump(result, f)
        # Append a comma unless it's the last item
        if index < len(results) - 1:
            f.write(',')
    f.write(']')  # End the JSON array
